In [1]:
#imports here
import os
import re
import time
import random
import logging
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC

In [4]:
logging.basicConfig(filename='logs/indeed_scrapingap.log', format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
## optional
# capabilities = webdriver.DesiredCapabilities.Chrome
# proxy.add_to_capabilities(capabilities)
# proxy_ip_port = {
#   "http": "http://128.199.97.42:80"
# }
# chrome_options = Options()
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument(f'proxy-server={proxy_ip_port}')

In [5]:
s=Service('./chromedriver.exe')
driver = webdriver.Chrome(service=s)
driver.maximize_window()

def get_url(position,number):
    template="https://cn.indeed.com/jobs?q={}&start={}"   # add &l={}     ||| [ &fromage=1 ] to filter data which was posted within 24 hours 
    url=template.format(position, number)
    return url

record = []

def main_scraper(position, pages):
    number = 0
    for page in tqdm(range(0, pages)):
        url = get_url(position, number)
        number += 10
        driver.get(url)
        try:
            close = driver.find_element(By.XPATH, '//button[contains(@class,"icl-CloseButton icl-Modal-close")]')
            close.click()
        except:
            pass
        try:
            close = driver.find_element(By.XPATH, '//button[contains(@class,"icl-CloseButton icl-Modal-close")]')
            close.click()
        except:
            pass
        try:
            close_cookies = driver.find_element(By.XPATH, '//button[contains(@id,"onetrust-accept-btn-handler")]')
            close_cookies.click()
        except:
            pass
        try:
            close = driver.find_element(By.XPATH, '//html')
            close.click()
        except:
            pass
        time.sleep(random.uniform(0.5, 1.1))
        jobs = driver.find_elements(By.XPATH,'//div[contains(@class,"job_seen_beacon")]')

        for job in jobs:
            job.location_once_scrolled_into_view
            job.click()
            time.sleep(random.uniform(7.5, 10.8))
            job_title = job.find_element(By.XPATH,'.//h2[contains(@class,"jobTitle")]').text.strip('new').strip()
            try:
                company=job.find_element(By.XPATH,'.//span[@class="companyName"]').text
            except:
                company='None'
            try:
                location = job.find_element(By.XPATH,'.//div[contains(@class,"companyLocation")]').text
            except AttributeError:
                location='None'
            try:
                salary= job.find_element(By.XPATH,'.//div[contains(@class,"salary-snippet-container")]').text
            except:
                salary = 'Negotiable'
            try:
                contract= job.find_element(By.XPATH,'.//div[contains(@class,"attribute_snippet")]').text
            except:
                contract = 'None'
            try:
                links = job.find_element(By.XPATH,'.//h2[contains(@class,"jobTitle")]/a').get_attribute('href')
            except AttributeError:
                links = 'None'
            try:
                description = driver.find_element(By.XPATH,'//div[@id = "jobDescriptionText"]').text.strip()
            except NoSuchElementException:
                description = 'None'
            data = {'Job_title': job_title, 'company':company,'location':location,'salary':salary, 'contract':contract, 'links':links, 'Full_Description': description}
            record.append(data)
    return record

position1='python developer'
location = 'china'
start_number = 2
data = main_scraper(position1, start_number)
data

100%|██████████| 2/2 [04:57<00:00, 148.57s/it]


[{'Job_title': 'EP RD System Test Developer-Python 上海金山',
  'company': 'Siemens',
  'location': '上海市',
  'salary': 'Negotiable',
  'contract': 'None',
  'links': 'https://cn.indeed.com/rc/clk?jk=803c09cd7c624f68&fccid=ea6bb53f0b18b8f2&vjs=3',
  'Full_Description': 'What are my responsibilities?\nBased on System, define system test plan and system integrated test solution or development test system.\nTest result analysis (by MATLAB or Scilab) to provide related proposals for design Optimization\nCoordinate automatic test platform development\nOptimize the existing digital solutions and make further updates accordingly to the development digitalization trends within and outside Siemens.\nWhat do I need to qualify for this job?\nProficiency in test management tools, familiar with Linux system basic instructions, experience to embedded system test platform\nBe familiar with Python.\nHands-on experience to build up the testing system platform\nGood communication skills\nBe able to read and 

In [7]:
df = pd.DataFrame(data)
tag = datetime.now().strftime("%Y-%m-%d-%H%M%S")
file_name = f"data/job-{tag}-{position1}-{start_number}_records-{location}.xlsx"
s_time = time.time()
df.to_excel(file_name, encoding='utf-8')

In [8]:
df

,Job_title,company,location,salary,contract,links,Full_Description
0,EP RD System Test Developer-Python 上海金山,Siemens,上海市,Negotiable,None,https://cn.indeed.com/rc/clk?jk=803c09cd7c624f...,What are my responsibilities?\nBased on System...
1,2023 Fresh Graduates for Artificial Intelligenc,Zf Friedrich,上海市,Negotiable,None,https://cn.indeed.com/rc/clk?jk=8329d1b4c9d7bf...,Job Description :\n\nWhat's Next Join ZF!\nZF ...
2,Internship - Software Developer,NOKIA,上海市,Negotiable,实习,https://cn.indeed.com/rc/clk?jk=019a6c59202444...,Come create the technology that helps the worl...
3,Backend Software Development Engineer 软件开发工程师,Mercedes-Benz Auto Finance Ltd.,北京市,Negotiable,None,https://cn.indeed.com/rc/clk?jk=0031b4be374746...,Aufgaben\nBackend Developer will take a key ro...
4,Developer Inter,Ericsson,广州市,Negotiable,实习,https://cn.indeed.com/rc/clk?jk=e2d94f7c8612e1...,Our Exciting Opportunity\nWe are now looking f...
5,Quantitative Researcher / Senior Quantitative ...,Bank of China,中国,Negotiable,None,https://cn.indeed.com/rc/clk?jk=27d979d56ef924...,Responsibilities:\nkdb+/q developer role in th...
6,Vocational Training Program Intern (Developer ...,SAP,上海市,Negotiable,实习,https://cn.indeed.com/rc/clk?jk=93081f8db36c9c...,We help the world run better\nOur company cult...
7,Vocational Training Program Intern (Developer ...,Hybris,上海市,Negotiable,实习,https://cn.indeed.com/rc/clk?jk=34d55a7293d058...,We help the world run better\nOur company cult...
8,SAP China iXp Intern - Cloud Native Intern Dev...,SAP,上海市,Negotiable,实习,https://cn.indeed.com/rc/clk?jk=07e38c50cca197...,We help the world run better\nOur company cult...
9,Web developer,MING Labs,苏州市,Negotiable,None,https://cn.indeed.com/rc/clk?jk=0e2a6887d695bd...,工作内容：\n基于公司战略发展，MING Labs正在寻找一位Web开发人员加入苏州团队。\...


## YaY its working 
## Now it can be modfied for any other country indeed portal 